In [1]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,519590,27.8,1163607,62.2,641871,34.3
Vcells,986491,7.6,8388608,64.0,1776327,13.6


Warning message:
“package ‘ROCR’ was built under R version 4.1.0”Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


ERROR: Error in file.choose(): selección de archivo cancelada


In [3]:
getwd()

[1] "/Users/achain/Documents/github/labo/src/02-Rpart"

In [2]:
#setwd("~/cloudL/UA/labo2022/")

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000

#Establezco semilla aleatoria
set.seed(500119)



In [3]:
#cargo los datos
dataset <- fread("./datasets/competencia1_2022.csv")

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

train_rows <- createDataPartition(dtrain$clase, p = .66, list = FALSE)

#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
dtest <- dtrain[-train_rows,]
dtrain <- dtrain[train_rows,]

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [4]:
#Funcion que calcula la ganancia para una prediccion y valores reales
fmetrica_ganancia_rpart  = function( probs, clases, pclase_valor_positivo )
{
 
  return(  sum(    (probs > 1/40 ) * 
                   ifelse( clases== pclase_valor_positivo, kTPGain, kFPGain )   
              )
         )
}

In [5]:
# Funcion que entrena el arbol y luego determina la ganancia sobre el set de test
modelo_rpart_ganancia = function( ptrain, ptest, pmaxdepth, pminbucket, pminsplit, pcp )
{

  modelo   <-  rpart("clase_ternaria ~ .",   data = ptrain,  xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)

  #aplico el modelo a datos nuevos
  testing_prediccion  <- predict( modelo, ptest)[, "BAJA+2"]
    
  return(fmetrica_ganancia_rpart(testing_prediccion,ptest[,..kclase_nomcampo],"BAJA+2"))
  
}

In [15]:
5+4+4+4


[1] 17

In [19]:
#Cuenta los modelos probados
linea <- 1

#Establece una ganancia inicial muy baja a superar
max_gan <- -100000000

#For anidados para grid search 
#for( vcp in c( 0.00001,0.01,-1)) #Complexity Factor
for( vcp in c( 0.000001)) #Complexity Factor

for( vmaxdepth in  c(6,10,18,22)) #max Depth
#for( vminsplit in  c(2, 4, 8, 10, 15, 20, 30, 50, 100, 150, 200, 300, 400 ) ) #Min Split
for( vminsplit in  c(18,80,200, 300, 400) ) #Min Split

#for( vminbucket  in  unique( as.integer(c(1,2,3,4,5,  vminsplit/10, vminsplit/5, vminsplit/3, vminsplit/2 )) ) ) #Min Bucket
for( vminbucket  in  unique( as.integer(c( vminsplit/5, vminsplit/3, vminsplit/2 )) ) ) #Min Bucket

{ 

    #Muestra los parametros a testear
    message(paste('Intentando',vcp,vminsplit,vminbucket,vmaxdepth))
    #Llama a la funcion que entrena el modelo y calcula la ganancia para esos parametros
    res <- modelo_rpart_ganancia( dtrain, dtest, 
                                  pmaxdepth=vmaxdepth, pminbucket=vminbucket, pminsplit=vminsplit, pcp=vcp )

  
    #Verifica si se supero la mejor ganancia hasta el momento. En caso de haberla superado graba los hiperparametros nuevos
    if (res>max_gan) {
      max_gan <- res
      message(paste("Nuevo Maximo: ", res*3, vcp,vminsplit,vminbucket,vmaxdepth))
      pcp<-vcp
      pminsplit<-vminsplit
      pminbucket<-vminbucket
      pmaxdepth<-vmaxdepth
  }
  #Muestra al ganancia de la iteracion
  message(paste('Ganancia:',res*3,'Ganancia Maxima',max_gan*3))
    
  linea <- linea+1


}

Intentando 1e-06 18 3 6
Nuevo Maximo:  17022000 1e-06 18 3 6
Ganancia: 17022000 Ganancia Maxima 17022000
Intentando 1e-06 18 6 6
Ganancia: 16440000 Ganancia Maxima 17022000
Intentando 1e-06 18 9 6
Ganancia: 16440000 Ganancia Maxima 17022000
Intentando 1e-06 80 16 6
Ganancia: 16440000 Ganancia Maxima 17022000
Intentando 1e-06 80 26 6
Ganancia: 16440000 Ganancia Maxima 17022000
Intentando 1e-06 80 40 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 200 40 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 200 66 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 200 100 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 300 60 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 300 100 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 300 150 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 400 80 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 400 133 6
Ganancia: 0 Ganancia Maxima 17022000
Intentando 1e-06 400 200 6
Ganancia: 0 Ganancia Max

In [28]:
max_gan

[1] 7370000

In [20]:
#Entrena el modelo con el dataset completo
t0       <-  Sys.time()
modelo   <-  rpart("clase_ternaria ~ .",   data = dataset,   xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

[1] 10.25113


In [23]:
colnames(dtrain)

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_personales"               
 [34] "mprestamos_personales"               
 [35] "cprestamos_prendarios"               
 [36] "mprestamos_prendarios"               
 [37] "cprestamos_hipotecarios"             
 [38] "mprestamos_hipotecarios"             
 [39] "cplazo_fijo"                         
 [40] "mplazo_fijo_dolares"                 
 [41] "mplazo_fijo_pesos"                   
 [42] "cinversion1"                         
 [43] "minversion1_pesos"                   
 [44] "minversion1_dolares"                 
 [45] "cinversion2"                         
 [46] "minversion2"                         
 [47] "cseguro_vida"                        
 [48] "cseguro_auto"                        
 [49] "cseguro_vivienda"                    
 [50] "cseguro_accidentes_personales"       
 [51] "ccaja_seguridad"                     
 [52] "cpayroll_trx"                        
 [53] "mpayroll"                            
 [54] "mpayroll2"                           
 [55] "cpayroll2_trx"                       
 [56] "ccuenta_debitos_automaticos"         
 [57] "mcuenta_debitos_automaticos"         
 [58] "ctarjeta_visa_debitos_automaticos"   
 [59] "mttarjeta_visa_debitos_automaticos"  
 [60] "ctarjeta_master_debitos_automaticos" 
 [61] "mttarjeta_master_debitos_automaticos"
 [62] "cpagodeservicios"                    
 [63] "mpagodeservicios"                    
 [64] "cpagomiscuentas"                     
 [65] "mpagomiscuentas"                     
 [66] "ccajeros_propios_descuentos"         
 [67] "mcajeros_propios_descuentos"         
 [68] "ctarjeta_visa_descuentos"            
 [69] "mtarjeta_visa_descuentos"            
 [70] "ctarjeta_master_descuentos"          
 [71] "mtarjeta_master_descuentos"          
 [72] "ccomisiones_mantenimiento"           
 [73] "mcomisiones_mantenimiento"           
 [74] "ccomisiones_otras"                   
 [75] "mcomisiones_otras"                   
 [76] "cforex"                              
 [77] "cforex_buy"                          
 [78] "mforex_buy"                          
 [79] "cforex_sell"                         
 [80] "mforex_sell"                         
 [81] "ctransferencias_recibidas"           
 [82] "mtransferencias_recibidas"           
 [83] "ctransferencias_emitidas"            
 [84] "mtransferencias_emitidas"            
 [85] "cextraccion_autoservicio"            
 [86] "mextraccion_autoservicio"            
 [87] "ccheques_depositados"                
 [88] "mcheques_depositados"                
 [89] "ccheques_emitidos"                 

In [27]:
dtrain[clase_ternaria,count()]

ERROR: Error in `[.data.table`(dtrain, clase_ternaria, count()): clase_ternaria is not found in calling scope and it is not a column of type logical. When the first argument inside DT[...] is a single symbol, data.table looks for it in calling scope.


In [13]:
#Detalles del modelo
modelo

n= 238608 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 238608 971.016000 0.004086200  
   2) ctrx_quarter>=18.5 220827 467.008200 0.002119306  
     4) ctrx_quarter>=49.5 185242 186.811200 0.001009490  
       8) ccomisiones_mantenimiento< 23 185241 185.813200 0.001004097 *
       9) ccomisiones_mantenimiento>=23 1   0.000000 1.000000000 *
     5) ctrx_quarter< 49.5 35585 278.781100 0.007896586  
      10) Master_mpagominimo< 706832.1 35584 277.796800 0.007868705 *
      11) Master_mpagominimo>=706832.1 1   0.000000 1.000000000 *
   3) ctrx_quarter< 18.5 17781 492.543600 0.028513580  
     6) mpasivos_margen>=34.305 8946 132.962800 0.015090540 *
     7) mpasivos_margen< 34.305 8835 356.336800 0.042105260  
      14) mprestamos_personales>=36166.5 1212   9.917492 0.008250825  
        28) mcuenta_corriente< 1586.375 1211   8.933113 0.007431874 *
        29) mcuenta_corriente>=1586.375 1   0.000000 1.000000000 *
      15) mprestamos_personales< 36166.5 7623 3

In [14]:
#Leo datos a estimar
dataset_pred <- fread(karchivo_predecir)


#Genero la clase
#dataset_pred[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
#dataset_pred[ ,  (kcampos_a_borrar) := NULL    ] 

prediccion  <- predict( modelo, dataset_pred ) #aplico el modelo

In [15]:
dataset_pred[ , prob_baja := prediccion]
dataset_pred[ , Predicted  := as.numeric(prob_baja > 0.025) ]

entrega  <- dataset_pred[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./salida_rpart_grid.csv", sep="," )